In [ ]:
using DynamicPolynomials
using SumOfSquares
using CSDP
using JuMP
using Plots
using LinearAlgebra
using DifferentialEquations
using PyCall

$$x_1' = x_2, x_2' = \frac{g}{l}(x_1 - \frac{x_1^3}{6}) + \frac{1}{ml^2} u $$ 

# 0. Init system variable and parameters

In [ ]:
#init state and input variables
@polyvar x[1:2]
@polyvar u[1:1]

#define system dynamics
vectorField = [ x[2], -x[1] + 9.8*(x[1]-(x[1]^3/6)) + u[1] ]

#define unsafe set (obstacle)
g = 0.04 - x[1]^2 - x[2]^2

#state and input bounds
bounds = [[-10., 10.], [-10., 10.] ]
u_bounds = [[-π,π]]

In [ ]:
function prepare_domain(x::Vector{<:Variable}, bounds::Vector{Vector{Float64}})
   poly_list = [ 
        @set(x[i] - l >= 0) ∩ @set(u - x[i] >= 0 ) ∩ @set((x[i] -l)*(u-x[i]) >= 0)
        for (i, (l, u)) in enumerate(bounds)
        ] 
   poly_list
end

In [ ]:
#instantiate parameters
λ = 1
ϵ = 1
max_degree=4
U = [[-1.0],[1.0]]

In [ ]:
# generate test points
function get_random(limits::Vector{Vector{Float64}}, g::Polynomial)
    function get_random_scalar(lb, ub )
        lb + rand()*(ub - lb) 
    end
    while (true)
        pt = [get_random_scalar(l[1], l[2]) for l in limits]
        if g(pt[1], pt[2]) >= 0
            continue
        else
            return pt
        end
    end
end

test_pts = [ get_random(bounds, g) for _ in 1:25];

# 1. Computing initial set of barriers for each input U_i

In [ ]:
# function to generate initial barriers for each input

function generate_barrier(x, u, bounds, g, vectorField, U, test_pts; max_degree=4,ϵ = 0.25, λ = 0.1, γ = 10.)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    
    B = polynomial(c[1:end], monos) 
    #negative inside the domain
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0) )
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=dom)
    
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), value(B_dot), lm
end

In [ ]:
# function to comupte transit time for each barrier 

function refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B, B_dot; ϵ = 0.25, κ = .1)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom_list_u = prepare_domain(u, u_bounds)
    dom_list = append!(dom_list, dom_list_u)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, η)
    @variable(model, δ >=0)
    dom3 = dom ∩ @set(B >= 0) ∩ @set(B <= κ)
    @constraint(model, -η*B - δ <= B_dot, domain=dom3)
    @constraint(model, η*κ + δ >= 0)
    @objective(model, Min, δ)
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    println("η = $(JuMP.value(η))")
    println("δ = $(JuMP.value(δ))")
    τ =  (κ)/max(JuMP.value(δ),JuMP.value(η) *  κ + JuMP.value(δ))
    println("τd = $(JuMP.value(τ))")
    return τ
end

In [ ]:
elapsed1 = @elapsed begin
B_0, B_0d, lm0 = generate_barrier(x, u, bounds, g, vectorField, U[1], test_pts; max_degree = 2)
end
display(B_0)
display(B_0d)
t_0d = refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B_0,  B_0d)

if (!ismissing(B_0))
    test_pts = filter!(pt -> B_0(pt...) <= 0., test_pts)
end

In [ ]:
elapsed2 = @elapsed begin
B_1, B_1d, lm1 = generate_barrier(x, u, bounds, g, vectorField, U[2], test_pts; max_degree=2)
end
display(B_1)
display(B_1d)
t_1d = refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B_1,  B_1d)

if (!ismissing(B_1))
    test_pts = filter!(pt -> B_1(pt...) <= 0., test_pts)
end

# 2. Computing Successive Barriers

In [ ]:
# functions to compute successive barriers

In [ ]:
function generate_successive_barrier(x::Vector{<:Variable}, u::Vector{<:Variable}, 
        bounds::Vector{Vector{Float64}}, g::Polynomial, 
        vectorField::Vector{<:Polynomial}, U::Vector{Float64}, 
        test_pts::Vector{Vector{Float64}}, ancestors::Vector{<:Polynomial}, 
        pt_to_eliminate::Vector{Float64}; 
        max_degree=4,ϵ = 1, λ = 1, γ = 10., κ = .1)
    solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")

    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    
    B = polynomial(c[1:end], monos) 
    # negative inside the obstacle
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0) )
    
    # dynamics constraints
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    if size(ancestors)[1] >= 1
        new_domain = dom ∩ (reduce(∩, [@set(B <= 0) for B in ancestors]))
    else
        new_domain = dom 
    end
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=new_domain)
    
    # eliminate the point we would like to eliminate
    @constraint(model, B(pt_to_eliminate...) >= ϵ)
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    
    # maximize the sum of values for all test points
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), value(B_dot), lm
end

In [ ]:
function refine_barrier_succ_dn(x, u, bounds, u_bounds, g, vectorField, B, B_dot, ancestors; ϵ = 0.25, κ = .1)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom_list_u = prepare_domain(u, u_bounds)
    dom_list = append!(dom_list, dom_list_u)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, η)
    @variable(model, δ>=0)

    if size(ancestors)[1] >= 1
        new_domain = dom ∩ (reduce(∩, [@set(b <= 0) for b in ancestors]))
    else
        new_domain = dom 
    end
    
    dom3 = new_domain ∩ @set(B >= 0) ∩ @set(B <= κ)
    @constraint(model, -η*B - δ <= B_dot, domain=dom3)
    @constraint(model, η*κ + δ >= 0)

    #set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    @objective(model, Min, δ)
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    println("η = $(value(η))")
    println("δ = $(value(δ))")
    τ =  (κ)/max(value(δ),value(η) *  κ + value(δ))
    println("τd = $(value(τ))")
    return τ
end

In [ ]:
function compute_next_level_barriers(x::Vector{<:Variable}, u::Vector{<:Variable}, bounds::Vector{Vector{Float64}},
                                    u_bounds::Vector{Vector{Float64}},
                                    g::Polynomial, vectorField::Vector{<:Polynomial}, 
                                    U::Vector{Vector{Float64}}, test_pts::Vector{Vector{Float64}}, 
                                    ancestors::Vector{<:Polynomial}) 
    eliminated = []
    second_level_barriers=[]
    for (j,pt) in enumerate(test_pts) 
        if j in eliminated
            # ignore the ones already eliminated
            continue
        end
        # go through each control 
        for u_val in U
            # generate a barrier 
            B = generate_successive_barrier(x, u, bounds, g, vectorField, u_val, test_pts, ancestors, pt)
            # check if we found something
            if !ismissing(B)
                println("Bingo: Found $B")
                useful = false # check if it is actually useful
                for (k,pt_new) in enumerate(test_pts)
                    if k in eliminated
                        continue
                    end
                   if (B[1](pt_new...) >= 0.)
                    push!(eliminated, k)   
                    println("\t eliminated $pt_new")
                    useful=true
                   end
                end
                if (useful)
                    println("Num remaining = $(size(test_pts)[1] - size(eliminated)[1])")
                    td = refine_barrier_succ_dn(x, u, bounds, u_bounds, g, vectorField, B[1],  B[2], ancestors)         
                    push!(second_level_barriers, (B[1], u_val, td, B[3]))
                    break
                end
            end
        end
    end
    new_test_pts = [pt for (j, pt) in enumerate(test_pts) if !(j in eliminated)]
    return (second_level_barriers, new_test_pts)
end

In [ ]:
zero_level_barriers::Vector{Polynomial} =[B_0, B_1]
print(size(zero_level_barriers))
s_elapsed = @elapsed begin
(first_level_barriers, test_pts_1) = compute_next_level_barriers(x, u, bounds, u_bounds, g, vectorField, U, test_pts, zero_level_barriers)
end

In [ ]:
test_pts_1 = [ get_random(bounds, g) for _ in 1:25];

In [ ]:
all_barriers1::Vector{Polynomial} = []
append!(all_barriers1, zero_level_barriers)
append!(all_barriers1, [B for (B, _) in first_level_barriers])
(second_level_barriers, test_pts_2) = compute_next_level_barriers(x, u, bounds, u_bounds, g, vectorField, U, test_pts_1, all_barriers1)

# 3. Plots

In [ ]:
# plotting functions

function circleShape(x,y, r)
    θ = LinRange(0, 2*π, 500)
    x .+ r*cos.(θ), y .+ r*sin.(θ)
end

function plot_ci_region2lev(limits::Tuple{Float64,Float64}, lev1_barriers::Vector{<:Polynomial}, lev2_barriers::Vector{<:Polynomial}; δ = 0.1, theta_val=0.0, filestem="ics")
    plot(xlims=limits, ylims=limits)
    rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])
    for x in limits[1]:δ:limits[2]
        for y in limits[1]:δ:limits[2]
            if (any([ B(x, y) > 0. for B in lev1_barriers]))
                plot!(rectangle(δ, δ, x-δ, y-δ), label=false, fill=:seagreen1, opacity=0.5,linecolor=:seagreen1)
            else
                if (any([ B(x, y) > 0. for B in lev2_barriers]))
                    plot!(rectangle(δ, δ, x-δ, y-δ), label=false, fill=:limegreen, linecolor=:limegreen)
                end 
            end
        end
    end
    plot!(circleShape(0,0,0.2), seriestype =[:shape], lw=0.5, c=:black, linecolor=:black, legend=false, aspectratio=1  )
    plot!(xlims=limits, ylims=limits)
    filename="figures/$filestem-theta-$(round(theta_val; digits=2)).png"
    savefig(filename)
    plot!(xlims=limits, ylims=limits)
end


In [ ]:
# improvement in the control invariant region at level 1
all_barriers = [B for (B,_) in first_level_barriers]
append!(all_barriers, [B_0, B_1]);
plot_ci_region2lev((-10., 10.), [B_0, B_1], all_barriers; δ=0.25, theta_val=-3.1415/2, filestem="Figure7c-invp-lev1")

In [ ]:
# improvement in the control invariant region at level 2
lev2_barriers::Vector{Polynomial} = [B for (B,_) in second_level_barriers]
plot_ci_region2lev((-10., 10.0), all_barriers, lev2_barriers; theta_val=3.14, δ=0.25, filestem="Figure7c-invp-lev2")

In [ ]:
# minimum transit time over all barriers
t0_min = minimum([t_0d, t_1d]);
t1=[]
for i in first_level_barriers
    if (!ismissing(i[3]))
        t1 = append!(t1,i[3])
    end
end
t1 = filter(x -> x > 0, t1)
t1_min = minimum(t1);
t2=[]
for i in second_level_barriers
    if (!ismissing(i[3]))
        t2 = append!(t2,i[3])
    end
end
t2 = filter(x -> x > 0, t2);
t2_min = minimum(t2);

println("τ_min = $(minimum([t0_min, t1_min, t2_min]))")

# 4. Benchmarks

In [ ]:
# generating data for Table 1 in the paper (row for current system)

In [ ]:
println("Time taken for B1: $(elapsed1+elapsed2)")
println("# barriers B1: $(size(zero_level_barriers))")
println("Time taken for B2: $(s_elapsed)")
println("# barriers B2: $(size(first_level_barriers))")

In [ ]:
# generating test points with fixed seed for evaluations (Table 2 in paper)

function generate_random_points_fixed_seed(limits::Vector{Vector{Float64}}, g::Polynomial)
     function get_random_scalar(lb, ub )
        Random.seed!(1234)
        lb + rand()*(ub - lb) 
     end
     while (true)
         pt = [get_random_scalar(l[1], l[2]) for l in limits]
         if g(pt[1], pt[2]) >= 0
             continue
         else
             return pt
         end
     end
end
test_pts_fixed_seed = [ get_random(bounds, g) for _ in 1:1000];

In [ ]:
# functions for FOSSIL comparison (Table 2 in paper)

py"""
import math
def eval_fossil_barrier(x0,x1):
    barrier = (-0.087137006223201752 + 0.096000000834465027 * pow((-0.55000001192092896 + (-1.3140000104904175 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (-0.67100000381469727 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (-0.55099999904632568 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (-0.46200001239776611 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (-0.38299998641014099 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (-0.25400000810623169 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1)))) + (-0.17200000584125519 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (-0.10300000011920929 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1)))) + (0.21799999475479126 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (0.41699999570846558 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1))))), 2) - 0.15099999308586121 * pow((-0.48100000619888306 + (-1.2940000295639038 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (-0.44100001454353333 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (-0.40900000929832458 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (-0.39800000190734863 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (-0.32499998807907104 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (-0.25200000405311584 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (0.33500000834465027 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (0.61100000143051147 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1)))) + (0.77600002288818359 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1)))) + (1.2990000247955322 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1))))), 2) - 0.7630000114440918 * pow((-0.39899998903274536 + (-0.61799997091293335 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1)))) + (-0.60399997234344482 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1)))) + (-0.54799997806549072 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1)))) + (-0.53600001335144043 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (-0.20600000023841858 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (0.27799999713897705 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (0.39100000262260437 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (0.50499999523162842 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (1.1189999580383301 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (1.7079999446868896 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1))))), 2) - 0.028000000864267349 * pow((-0.35600000619888306 + (-1.1799999475479126 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (-0.68199998140335083 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1)))) + (-0.42300000786781311 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (-0.34099999070167542 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (-0.10000000149011612 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1)))) + (0.14000000059604645 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (0.19200000166893005 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (0.23299999535083771 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (0.43500000238418579 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1)))) + (0.83300000429153442 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1))))), 2) - 0.043000001460313797 * pow((-0.22100000083446503 + (-1.1460000276565552 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (-0.33799999952316284 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (-0.23100000619888306 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (-0.21600000560283661 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1)))) + (-0.12399999797344208 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (-0.083999998867511749 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (-0.081000000238418579 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1)))) + (0.010999999940395355 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (0.23100000619888306 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (0.45399999618530273 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1))))), 2) + 0.10000000149011612 * pow((-0.052000001072883606 + (-0.66600000858306885 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (-0.43000000715255737 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (-0.22699999809265137 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1)))) + (-0.18700000643730164 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1)))) + (-0.13600000739097595 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1)))) + (-0.10199999809265137 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (-0.057999998331069946 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (0.14000000059604645 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (0.16200000047683716 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (0.55299997329711914 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1))))), 2) - 0.2460000067949295 * pow((0.043000001460313797 + (-0.7850000262260437 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (-0.5820000171661377 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (-0.52499997615814209 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (-0.37999999523162842 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (0.20100000500679016 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (0.38499999046325684 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1)))) + (0.49399998784065247 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1)))) + (0.54400002956390381 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (0.71700000762939453 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1)))) + (0.92100000381469727 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1))))), 2) + 0.19799999892711639 * pow((0.19699999690055847 + (-0.41999998688697815 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (-0.40900000929832458 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1)))) + (-0.3449999988079071 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (-0.20399999618530273 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (0.05299999937415123 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (0.15199999511241913 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1)))) + (0.18999999761581421 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (0.30500000715255737 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (0.62199997901916504 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (0.65600001811981201 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1))))), 2) + 0.13899999856948853 * pow((0.66399997472763062 + (-0.29300001263618469 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (-0.11299999803304672 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (-0.078000001609325409 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (0.026000000536441803 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (0.3880000114440918 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (0.4830000102519989 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (0.49900001287460327 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1)))) + (0.5910000205039978 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (0.65700000524520874 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1)))) + (0.83300000429153442 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1))))), 2) + 0.68900001049041748 * pow((0.80199998617172241 + (-0.72500002384185791 / (1 + math.exp((-0.2070000022649765 + 1.1920000314712524 * x0 - 0.50400000810623169 * x1)))) + (-0.42599999904632568 / (1 + math.exp((2.875 + 0.39899998903274536 * x0 - 0.17299999296665192 * x1)))) + (-0.1940000057220459 / (1 + math.exp((2.7920000553131104 + 1.5529999732971191 * x0 + 0.17599999904632568 * x1)))) + (0.44499999284744263 / (1 + math.exp((1.4589999914169312 + 0.59299999475479126 * x0)))) + (0.55199998617172241 / (1 + math.exp((-0.65600001811981201 - 1.0729999542236328 * x0 - 0.24300000071525574 * x1)))) + (0.60000002384185791 / (1 + math.exp((0.008999999612569809 - 1.2779999971389771 * x0 - 0.31799998879432678 * x1)))) + (0.66699999570846558 / (1 + math.exp((3.369999885559082 + 0.88300001621246338 * x0 - 0.10000000149011612 * x1)))) + (0.77399998903274536 / (1 + math.exp((0.66900002956390381 - 1.2359999418258667 * x0 + 0.72100001573562622 * x1)))) + (0.81000000238418579 / (1 + math.exp((-0.49900001287460327 - 1.5859999656677246 * x0 - 0.032000001519918442 * x1)))) + (1.809999942779541 / (1 + math.exp((3.25 + 0.91299998760223389 * x0 - 0.99199998378753662 * x1))))), 2))
    return barrier
"""
fossil_eval = py"eval_fossil_barrier";

function B_lex(state, zero_level_barriers, one_level_barriers, two_level_barriers)
   b0, u0 = maximum([ (B(state...), u) for (B, u) in zero_level_barriers])
   if (b0 >= 0)
        return b0, u0
    end
   b1, u1 = maximum([ (B(state...), u) for (B, u) in one_level_barriers])
   if (b1 >= 0)
        return b1, u1
    end
   b2, u2 = maximum([ (B(state...), u) for (B, u) in two_level_barriers])
   if (b2 >= 0)
        return b2, u2
   end
   return max((b0, u0), (b1, u1), (b2, u2))
end

function fossil_comparisons(zero_level_barriers,first_level_barriers,second_level_barriers, fossil_eval, test_pts_fixed_seed)
    counter = 0
    counter_f = 0
    counter_s = 0
    counter_x = 0
    for pt in test_pts_fixed_seed
        c,_ = B_lex(pt, zero_level_barriers, first_level_barriers, second_level_barriers )
        if (fossil_eval(pt[1],pt[2]))>=0 && -c < 0
            counter = counter + 1
        end
        if (fossil_eval(pt[1],pt[2]))<0
            counter_f = counter_f + 1
        end
        if (-c)<0
            counter_s = counter_s + 1
        end
        if (fossil_eval(pt[1],pt[2]))<0 && -c >= 0
            counter_x = counter_x + 1
        end
    end
    println("Total test points: 1000")
    println("Safe in FOSSIL: $(counter)")
    println("Safe in Ours: $(counter_s)")
    println("Unsafe in FOSSIL and Safe in Ours: $(counter_f)")
    println("Safe in FOSSIL and Unsafe in Ours: $(counter_x)")
    
    return counter_f, counter_s, counter, counter_x
end

In [ ]:
fossil_comparisons(zero_level_barriers,first_level_barriers,second_level_barriers, fossil_eval, test_pts_fixed_seed)

# 5. Verification

In [ ]:
"""
Certifying barriers using constraint PSD check
"""

using DelimitedFiles

function check_psd_constraints(multipliers_array::Vector)
    counter = 0
    M = []
    for mults in multipliers_array
        for lms in mults
            for lm in lms
                if !isposdef(Matrix(lm.Q)) && !isposdef(Diagonal(svd(Matrix(lm.Q)).S))
                    counter += 1
                else 
#                     display(Matrix(lm.Q))
                    push!(M, [Matrix(lm.Q)])
                end
            end
        end
    end
    open("matrices_ip.txt", "w") do io
        writedlm(io, M)
    end
    if counter==0
        println("All constraints are PSD: Barrier certified")
    else
        println("PSD check failed!")
    end
end

In [ ]:
all_barrier_plus = push!([first_level_barriers], second_level_barriers);

In [ ]:
lms=[]
for i in all_barrier_plus
    if (!ismissing(i[1][4]))
        lms = append!(lms,i[1][4])
    end
end

In [ ]:
check_psd_constraints([lm0,lm1,lms])